In [34]:
import sys
import os

# Finn prosjektroten basert på notebookens plassering
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(os.path.join(project_root, 'src'))

# Importer funksjonene
from data_visualizations.dataframes import build_dataframe_from_json, build_dataframe_from_db

json_df = build_dataframe_from_json('data/clean/cleaned_data_nilu.json')
db_df = build_dataframe_from_db('data/clean/frost.db', 'weather_data')


# 📊 Data visualisering og analyse

Denne notebooken bygger videre på data vi har hentet og renset fra både **Frost** og **NILU**. Vi skal nå starte analysen og visualiseringen av dataen og forsøke å trekke ut meningsfull innsikt. Vi vil bruke visuelle fremstillinger for å forstå variasjoner, sammenhenger og trender i miljødataen. Først skal vi se luftkvaliteten i Trondheim før vi går videre på været.

Målet er å identifisere mønstre som er interessante i seg selv (f.eks. hvordan luftkvalitet varierer gjennom året), men også å gi et godt kunnskaps og sammenlignings-grunnlag for å forstå den prediktive analysen i neste steg av prosjektet.

### 🌫️ - Luftkvaliteten i Trondheim
Først skal vi få oversikt over statistiske verdier som gjennomsnitt, median og standaravvik over ulike luftkvalitets indikatorer, som gir oss en rask oversikt over dataens fordeling, variasjon og typiske verdier. Dette hjelper oss med å forstå dataen og identifisere eventuelle avvik eller mønstre.

**DataFrame over luftkvalitet**

In [35]:
print(json_df)

     dateTime        NO2       PM10      PM2.5  generated_NO2  generated_PM10  generated_PM2.5
0  2010-01-01  25.720800  12.471300   7.697400       0.000000        1.000000         1.000000
1  2010-01-02  26.260400  21.662750   9.579350       0.000000        0.500000         1.000000
2  2010-01-03  21.947200  17.682100   9.237633       0.000000        0.333333         0.666667
3  2010-01-04  29.279167  21.415267  13.589900       0.000000        0.000000         0.333333
4  2010-01-05  36.163900  19.738867  15.021700       0.000000        0.000000         0.333333
5  2010-01-06  51.491700  23.242733  17.325633       0.000000        0.333333         0.666667
6  2010-01-07  41.849400  18.567533  12.385967       0.333333        0.666667         1.000000
7  2010-01-08  44.688267  22.191133  12.510233       0.333333        0.666667         1.000000


Videre ønsker vi å se på datasettet i sin helhet. Til dette har vi brukt plotly for å lage en interaktiv graf, hvor man selv enkelt kan filtrere ut enkelte verdier, skalere tidslinjen og lese av nøyaktige verdier for hver observasjon.

In [36]:
# Hente inn ploty over NILU

Litt om en annen visualiseringen

In [37]:
# Hente inn visualiseringen nevnt overfor

### 🌦️ - Været i Trondheim
Nå går vi fra luftkvalitet til værdata og skal undersøke været i Trondheim de siste årene. Som med luftkvaliteten starter vi med å analysere statistikken for å få en god oversikt over dataene. 

In [38]:
# Hente inn dataframe for Frost
print(db_df)

  referenceTime  mean_air_temperature  mean_wind_speed  total_precipitation  generated_mean_air_temperature  generated_mean_wind_speed  generated_total_precipitation
0    2010-01-01                 -10.0              1.6                  2.1                               0                          0                              0
1    2010-01-02                 -13.5              2.6                  0.1                               0                          0                              0
2    2010-01-03                  -2.3              3.0                  1.1                               0                          0                              0
3    2010-01-04                  -4.7              2.8                  7.9                               0                          0                              0
4    2010-01-05                 -15.8              2.7                  0.4                               0                          0                              0
5   